In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
import time
import random
import numpy as np
import scipy.stats as st
import plotly.express as px

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from game_simulation import CoinGameSimulation

Cheaters are only biased against heads!

In [2]:
def simulate_strategy(strategy, n_simulations=500):
    # Simulate the game with the given strategy
    # strategy is a function that takes n_heads, n_tails and flips_left and returns an action in ("fair", "cheater", "one_flip", "five_flips")
    g = CoinGameSimulation()
    scores = []
    n_labels_list = []
    n_flips_list = []
    n_flips_per_label_list = []
    for _ in range(n_simulations):
        g.reset_game()
        n_labels = 0
        n_flips = 0
        n_flips_per_label = 0
        while not g.game_over:
            data = g.get_data()
            n_heads, n_tails, flips_left = data['heads'], data['tails'], data['flips_left']
            action = strategy(n_heads, n_tails, flips_left)

            if action == "fair":
                g.label_fair()
                n_labels += 1
                n_flips_per_label_list.append(n_flips_per_label)
                n_flips_per_label = 0
            elif action == "cheater":
                g.label_cheater()
                n_labels += 1
                n_flips_per_label_list.append(n_flips_per_label)
                n_flips_per_label = 0
            elif action == "one_flip":
                g.flip_one_coin()
                n_flips_per_label += 1
                n_flips += 1
            elif action == "five_flips":
                g.flip_five_coins()
                n_flips_per_label += 5
                n_flips += 5
            else:
                raise ValueError("Unknown action: {}".format(action))
        scores.append(data['score'])
        n_labels_list.append(n_labels)
        n_flips_list.append(n_flips)
    
    for name, l in zip(["score", "n_labels", "n_flips", "n_flips_per_label"],[scores, n_labels_list, n_flips_list, n_flips_per_label_list]):
        print(name, np.mean(l), "±", np.std(l))
    return scores

Random choice gives a score of around 7

In [3]:
_ = simulate_strategy(lambda n_heads, n_tails, flips_left: random.choice(["cheater", "fair"]))

score 7.642 ± 7.913901439871488
n_labels 15.164 ± 11.875399109082608
n_flips 0.0 ± 0.0
n_flips_per_label 0.0 ± 0.0


In [4]:
z = st.norm.ppf(.90)
p_fair = 0.5
p_cheater = 0.9

# https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval
def strategy1(n_heads, n_tails, flips_left):
    n = n_heads + n_tails
    if n == 0:
        return "five_flips"
    p_hat = n_heads / n

    interval = z * np.sqrt(p_hat * (1 - p_hat) / n)

    #print(n_heads, n_tails, flips_left)
    #print(p_hat - interval, p_hat + interval, 2*interval)


    if p_hat - interval < p_fair < p_cheater + interval:
        return "fair"
    
    if p_hat - interval < p_cheater < p_cheater + interval:
        return "cheater"
    
    if flips_left <= 0:
        if abs(p_fair-p_hat) > abs(p_cheater-p_hat):
            return "fair"
        else:
            return "cheater"
    
    return "one_flip"

_ = simulate_strategy(strategy1, n_simulations=500)

score 10.942 ± 11.005391224304567
n_labels 15.874 ± 13.749404496195462
n_flips 138.542 ± 96.60470090011148
n_flips_per_label 8.727604888496913 ± 16.978315817064725


In [5]:
# Vibe
def strategy2(n_heads, n_tails, flips_left):
    n_throws = n_heads + n_tails
    diff = (n_heads-n_tails)
    if diff > 5:
        return "cheater"
    elif diff < 0 or (n_throws > 10 and diff < 1):
        return "fair"
    if flips_left <= 0:
        if diff > 4:
            return "cheater"
        else:
            return "fair"
    return "one_flip"

random.seed(1)
_ = simulate_strategy(strategy2, n_simulations=1000)

score 32.969 ± 42.746555872958936
n_labels 44.387 ± 53.704164000568895
n_flips 267.313 ± 314.43099565882494
n_flips_per_label 6.022326356816185 ± 6.036929618535373


In [ ]:
#https://en.wikipedia.org/wiki/Checking_whether_a_coin_is_fair